# Creating L3 VPN Network

FABRIC provides a layer 3 IP networking services across every FABRIC site L3VPN. 

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config();
except Exception as e:
    print(f"Exception: {e}")

## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

slice_name = 'MySlice-l3vpn'
#[site1,site2] = fablib.get_random_sites(count=2)
site1='RENC'
site2='UKY'
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

network1_name='net1'
network2_name='net2'
network3_name='net3'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'

node1_subnet = '192.168.10.0/24'
# Only used with dedicated NICs
node1_vlan = "1001"

network1 = IPv4Network(node1_subnet)
network1_hosts = list(network1.hosts())

node2_subnet = '192.168.20.0/24'
# Only used with dedicated NICs
node2_vlan = "1002"

network2 = IPv4Network(node2_subnet)
network2_hosts = list(network2.hosts())

#model = 'NIC_Basic'
model = 'NIC_ConnectX_5'


In [ ]:
from fim.slivers.capacities_labels import Labels
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model=model, name=node1_nic_name).get_interfaces()[0]
    fim_iface1 = iface1.get_fim_interface()
    
    if model != 'NIC_Basic':
        # Pass VLAN only with Dedicated NICs
        fim_iface1.labels = Labels.update(fim_iface1.labels, ipv4_subnet=f'{network1_hosts[0]}/{network1.prefixlen}', vlan=node1_vlan)
    else:
        fim_iface1.labels = Labels.update(fim_iface1.labels, ipv4_subnet=f'{network1_hosts[0]}/{network1.prefixlen}')
    
    #peering_labels = Labels(ipv4_subnet=f'{network1_hosts[1]}/{network1.prefixlen}', asn='654321',  bgp_key='somesecret')
    #fim_iface1.peer_labels = peering_labels    
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    iface2  = node2.add_component(model=model, name=node2_nic_name).get_interfaces()[0]
    fim_iface2 = iface2.get_fim_interface()
    if model != 'NIC_Basic':
        # Pass VLAN only with Dedicated NICs
        fim_iface2.labels = Labels.update(fim_iface2.labels, ipv4_subnet=f'{network2_hosts[0]}/{network2.prefixlen}', vlan=node2_vlan)
    else:
        fim_iface2.labels = Labels.update(fim_iface2.labels, ipv4_subnet=f'{network2_hosts[0]}/{network2.prefixlen}')

    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1, iface2], type='L3VPN')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Observe the Slice's Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
node1 = slice.get_node(name=node1_name)
interface1 = node1.get_interface(network_name=network1_name)
print(interface1)
print(interface1.get_os_interface())

##  Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on there new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Then set a route from *this network* to the *other network* through the specified gateway.


Optionally, use the `node.execute()` method to show the results of adding the IP address and route.

### Configure Node1

Configure the interface on node1.  

In [ ]:
if model != 'NIC_Basic':
    try:
        node1 = slice.get_node(name=node1_name)   
        interface1 = node1.get_interface(network_name=network1_name)
        stdout, stderr = node1.execute(f'sudo ip link add link eth1 name {interface1.get_os_interface()} type vlan id {node1_vlan} ')
        stdout, stderr = node1.execute(f'sudo ip link set dev {interface1.get_os_interface()} up ')
        stdout, stderr = node1.execute(f'sudo ip addr add {network1_hosts[2]}/{network1.prefixlen} dev {interface1.get_os_interface()}  ')
        stdout, stderr = node1.execute(f'sudo ip route add {network2} via {network1_hosts[0]}  ')

        stdout, stderr = node1.execute(f'sudo ip addr show ')
        stdout, stderr = node1.execute(f'sudo ip route list')
    except Exception as e:
        print(f"Exception: {e}")
else:
    try:
        node1 = slice.get_node(name=node1_name)   
        interface1 = node1.get_interface(network_name=network1_name)
        stdout, stderr = node1.execute(f'sudo ip addr add {network1_hosts[2]}/{network1.prefixlen} dev {interface1.get_os_interface()}  ')
        stdout, stderr = node1.execute(f'sudo ip route add {network2} via {network1_hosts[0]}  ')

        stdout, stderr = node1.execute(f'sudo ip addr show ')
        stdout, stderr = node1.execute(f'sudo ip route list')
    except Exception as e:
        print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node and a route to the first network.

In [ ]:
if model != 'NIC_Basic':
    try:
        node2 = slice.get_node(name=node2_name)   
        interface2 = node2.get_interface(network_name=network1_name)
        stdout, stderr = node2.execute(f'sudo ip link add link eth1 name {interface2.get_os_interface()} type vlan id {node2_vlan} ')
        stdout, stderr = node2.execute(f'sudo ip link set dev {interface2.get_os_interface()} up ')
        stdout, stderr = node2.execute(f'sudo ip addr add {network2_hosts[2]}/{network2.prefixlen} dev {interface2.get_os_interface()}  ')
        stdout, stderr = node2.execute(f'sudo ip route add {network1} via {network2_hosts[0]}  ')

        stdout, stderr = node2.execute(f'sudo ip addr show ')
        stdout, stderr = node2.execute(f'sudo ip route list')
    except Exception as e:
        print(f"Exception: {e}")
else:
    try:
        node2 = slice.get_node(name=node2_name)   
        interface2 = node2.get_interface(network_name=network1_name)
        stdout, stderr = node2.execute(f'sudo ip addr add {network2_hosts[2]}/{network2.prefixlen} dev {interface2.get_os_interface()}  ')
        stdout, stderr = node2.execute(f'sudo ip route add {network1} via {network2_hosts[0]}  ')

        stdout, stderr = node2.execute(f'sudo ip addr show ')
        stdout, stderr = node2.execute(f'sudo ip route list')
    except Exception as e:
        print(f"Exception: {e}")

## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {network2_hosts[2]}')
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")